In [1]:
addprocs(4);

In [2]:
###########################################################################################

@everywhere using latent_model
using LineSearches, Optim, JLD, ForwardDiff, Pandas
using global_functions, initialize_latent_model, initialize_spike_obs_model
@everywhere using sample_model, PyPlot

In [3]:
###########################################################################################
## External inputs

ratname = "T036"
model_type_dir = "spikes"
path=ENV["HOME"]*"/Dropbox/spike-data_latent-accum";

In [4]:
###########################################################################################
## Constants

model_type = model_type_dir
map_str = "exp"
dt = 2e-2;
dimy = 4;
noise = "Poisson";

In [5]:
###########################################################################################
## Define a closure over optimization function

@everywhere ll(x) = ll_wrapper(x, p_const, fit_vec, data, model_type,
    beta_y=beta_y, mu0_y=mu0_y, noise=noise, n=103);

In [6]:
###########################################################################################
## Load data

sessid = [157201];

In [7]:
data,N = make_data(path,sessid,ratname;dt=dt,organize="by_trial");

In [8]:
p_y_star = map(i->[1.,10.,0.1,0.],1:N);
p_z_star = [10.,      0.,         10., -2.,        40.,    1e-6,      1.,    0.2];

In [9]:
kwargs = ((:py,p_y_star))
sampled_dataset!(data, p_z_star, rng = 2, num_reps=10, noise=noise; kwargs);

In [10]:
n = 203

203

In [11]:
#break up latent variables
vari,inatt,B,lambda_z,vara = p_z_star[1:5]

# spatial bin centers, width and edges
xc,dx,xe = bins(B,n=n)

# make initial latent distribution
P = P0(vari,inatt,n,dx,xc);

# build empty transition matrix
M = zeros(Float64,n,n);
# build state transition matrix for no input time bins
Mprime!(M,vara*dt,lambda_z,zero(Float64),dx,xc,n=n)

lambda = map(p->my_sigmoid(xc,p),p_y_star);

In [ ]:
@time output = -sum(pmap((L,R,T,nL,nR,N,SC) -> LL_single_trial(p_z_star, P, M, dx, xc, model_type,
        L, R, T, nL, nR,
        lambda=lambda[N],spike_counts=SC,
        comp_posterior=false,n=n,noise=noise),
        data["leftbups"], data["rightbups"], data["nT"], data["binned_leftbups"], 
        data["binned_rightbups"],
        data["N"],data["spike_counts"]))

In [ ]:
@time @parallel (+) for i = 1:length(data["T"]) 
    -LL_single_trial(p_z_star, P, M, dx, xc, model_type,
        data["leftbups"][i], data["rightbups"][i], data["nT"][i], 
        data["binned_leftbups"][i], data["binned_rightbups"][i],
        lambda=lambda[data["N"][i]],spike_counts=data["spike_counts"][i],
        comp_posterior=false,n=n,noise=noise);
end

In [ ]:
16